# Project - Airline AI Assistant


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "You are travelling only to UK, France, Japan and Germany"
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL, 
        messages = messages)
    return response.choices[0].message.content

# gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

In [6]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
travelling_dates = {"london": "Monday-Friday", "paris": "Only Sunday", "tokyo": "Monday-Friday", "berlin": "Weekends Only"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def get_travelling_dates(destination_city):
    print(f"Tool get_travelling_dates called for {destination_city}")
    city = destination_city.lower()
    return travelling_dates.get(city, "Unknown")

In [7]:
price_function = {
    "type": "function",
    "function": {
        "name": "get_ticket_price",
        "description": "Get the price of a return ticket to the destination city. Call this whenever a customer asks for the ticket price.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

Travelling_date_function = {
    "type": "function",
    "function": {
        "name": "get_travelling_dates",
        "description": "Get the travelling dates to the destination city. Call this whenever a customer asks for available days/flights.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

In [8]:
TOOLS = [price_function, Travelling_date_function]
PY_FUNCTIONS = {
    "get_ticket_price": get_ticket_price,
    "get_travelling_dates": get_travelling_dates,
}

In [9]:
SYSTEM_PROMPT = """You are a helpful travel agent.
- If user asks about ticket prices or available days for a city, call the matching tool with the city name.
- Combine results nicely if both price and dates are requested.
- If a city is unsupported, say so and suggest supported cities (London, Paris, Tokyo, Berlin).
"""

def build_messages(history, user_msg):
    msgs = [{"role": "system", "content": SYSTEM_PROMPT}]
    for h_user, h_assistant in history:
        if h_user:
            msgs.append({"role": "user", "content": h_user})
        if h_assistant:
            msgs.append({"role": "assistant", "content": h_assistant})
    msgs.append({"role": "user", "content": user_msg})

    # msgs = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": user_msg}]
    return msgs

In [10]:
def call_model_with_tools(messages):

    ## Safety cap to avoid infinite loops in case of mis-config
    for _ in range(5):
        resp = openai.chat.completions.create(
            model = MODEL,
            messages = messages,
            tools = TOOLS,
            tool_choice = "auto",
            temperature = 0.5,
        )
        msg = resp.choices[0].message

        ## If the model requested one or more tools
        if msg.tool_calls:
            # Append the assistant message that *contains* the tool_calls
            messages.append({
                "role": "assistant",
                "content": msg.content or "",
                "tool_calls": [tc.model_dump() for tc in msg.tool_calls],
            })

            # For each tool_call, run the corresponding Python function and send back a tool message
            for tool_call in msg.tool_calls:
                name = tool_call.function.name
                args = tool_call.function.arguments
                try:
                    parsed = json.loads(args) if isinstance(args, str) else args
                except Exception:
                    parsed = {}

                fn = PY_FUNCTIONS.get(name)
                if fn is None:
                    tool_output = f"Unknown tool: {name}"
                else:
                    # Ensure destination_city is present
                    dest = parsed.get("destination_city", "") if isinstance(parsed, dict) else ""
                    tool_output = fn(dest)

                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": name,
                    "content": json.dumps({"result": tool_output}),
                })

            continue

        ## No tool calls -> final assistant answer
        content = msg.content or ""
        messages.append({"role": "assistant", "content": content})
        return content

    return "Sorry, I couldn't complete that request."

In [11]:
def respond(user_message, history):
    messages = build_messages(history, user_message)
    final_text = call_model_with_tools(messages)
    return final_text

In [12]:
demo = gr.ChatInterface(
    fn = respond,
    title = "Travel Chat (Prices & Dates)",
    # description="Ask things like: 'How much is a ticket to Tokyo?' or 'When can I fly to Berlin?'.",
    cache_examples=False,
)
demo.launch()

/Users/thilina/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
